# Import

In [306]:
from mip import Model, xsum, maximize, BINARY 
import pandas as pd

# Data

In [307]:
data = pd.read_csv('knapsack.txt', sep=" ", names=['Item', 'Price', 'Weigth', 'Volume'])
data

,Item,Price,Weigth,Volume
0,Laptop,10,0.50,0.20
1,Flashcard,5,0.01,0.02
2,Cryptokey,1,0.01,0.02
3,Lamp,1,5.00,1.00
4,Battery,1,0.05,0.06
5,Charger,0,0.01,0.06


In [308]:
dependencies = {}
with open('dependencies.txt') as f:
    for line in f:
        curr_line = line.split()
        dependencies[curr_line[-1]] = curr_line[:-1]

In [309]:
dependencies

{'Cryptokey': ['Flashcard'],
 'Flashcard': ['Laptop'],
 'Laptop': ['Battery', 'Charger']}

In [310]:
oversized = {}
with open('oversized.txt') as f:
    for line in f:
        curr_line = line.split()
        oversized[curr_line[0]] = float(curr_line[1])

In [311]:
oversized

{'Laptop': 0.05, 'Lamp': 0.01}

# Simple knapsack

In [312]:
def simple_knapsack(c, data):
    p = list(data['Price'])
    w = list(data['Weigth'])
    I = range(len(w))

    m = Model('knapsack')

    x = [m.add_var(var_type=BINARY) for i in I]
    m.objective = maximize(xsum(p[i] * x[i] for i in I))
    m += xsum(w[i] * x[i] for i in I) <= c
    m.optimize()

    selected = [i for i in I if x[i].x >= 0.99]
    print('maximum weigth: {}'.format(c))
    print("selected items: {}".format(", ".join(data[data.index.isin(selected)].Item.to_list())))

In [313]:
data

,Item,Price,Weigth,Volume
0,Laptop,10,0.50,0.20
1,Flashcard,5,0.01,0.02
2,Cryptokey,1,0.01,0.02
3,Lamp,1,5.00,1.00
4,Battery,1,0.05,0.06
5,Charger,0,0.01,0.06


In [314]:
simple_knapsack(0.01, data)

maximum weigth: 0.01
selected items: Flashcard


In [315]:
simple_knapsack(0.02, data)

maximum weigth: 0.02
selected items: Flashcard, Cryptokey


In [316]:
simple_knapsack(0.07, data)
simple_knapsack(0.49, data)

maximum weigth: 0.07
selected items: Flashcard, Cryptokey, Battery
maximum weigth: 0.49
selected items: Flashcard, Cryptokey, Battery


In [317]:
simple_knapsack(0.5, data)

maximum weigth: 0.5
selected items: Laptop


In [318]:
simple_knapsack(float('Inf'), data)

maximum weigth: inf
selected items: Laptop, Flashcard, Cryptokey, Lamp, Battery


# Knapsack problem with dependencies

In [319]:
def deps_knapsack(c, data, dependencies):
    p = list(data['Price'])
    w = list(data['Weigth'])
    I = range(len(w))

    m = Model('knapsack')

    x = [m.add_var(var_type=BINARY) for i in I]
    m.objective = maximize(xsum(p[i] * x[i] for i in I))
    m += xsum(w[i] * x[i] for i in I) <= c

    for key, value in dependencies.items():
        m += xsum(x[i] for i in data[data.Item.isin(value)].index.to_list()) >= x[data[data.Item == key].index.to_list()[0]]

    m.optimize()

    selected = [i for i in I if x[i].x >= 0.99]
    print('maximum weigth: {}'.format(c))
    print("selected items: {}".format(", ".join(data[data.index.isin(selected)].Item.to_list())))

In [320]:
data

,Item,Price,Weigth,Volume
0,Laptop,10,0.50,0.20
1,Flashcard,5,0.01,0.02
2,Cryptokey,1,0.01,0.02
3,Lamp,1,5.00,1.00
4,Battery,1,0.05,0.06
5,Charger,0,0.01,0.06


In [321]:
dependencies

{'Cryptokey': ['Flashcard'],
 'Flashcard': ['Laptop'],
 'Laptop': ['Battery', 'Charger']}

In [322]:
# we can not take Flashcard or Cryptokey because of dependencies
deps_knapsack(0.04, data, dependencies)

maximum weigth: 0.04
selected items: 


In [323]:
# only Battery is acceptable at 0.05 until 0.51 (Charger + Laptop = 10)
deps_knapsack(0.05, data, dependencies)
deps_knapsack(0.5, data, dependencies)
deps_knapsack(0.51, data, dependencies)

maximum weigth: 0.05
selected items: Battery
maximum weigth: 0.5
selected items: Battery
maximum weigth: 0.51
selected items: Laptop, Charger


In [324]:
deps_knapsack(0.52, data, dependencies)
deps_knapsack(0.53, data, dependencies)
deps_knapsack(0.57, data, dependencies)

maximum weigth: 0.52
selected items: Laptop, Flashcard, Charger
maximum weigth: 0.53
selected items: Laptop, Flashcard, Cryptokey, Charger
maximum weigth: 0.57
selected items: Laptop, Flashcard, Cryptokey, Battery


In [325]:
deps_knapsack(float('Inf'), data, dependencies)

maximum weigth: inf
selected items: Laptop, Flashcard, Cryptokey, Lamp, Battery, Charger


# Knapsack problem with oversize items

In [326]:
def vol_knapsack(c, data, oversized):
    p = list(data['Price'])
    w = list(data['Weigth'])
    v = list(data['Volume'])
    I = range(len(w))

    m = Model('knapsack')

    x = [m.add_var(var_type=BINARY) for i in I]

    z = {}
    for key in oversized:
        z[key] = [m.add_var(var_type=BINARY) for i in I]

    m.objective = maximize(xsum(p[i] * x[i] for i in I))
    m += xsum(w[i] * x[i] for i in I) <= c

    ind_over = 0
    vol_over = 0.05

    for key, vol_over in oversized.items():
        ind_over = data[data.Item == key].index.to_list()[0]
        for i in I:
            m += z[key][i] <= x[ind_over]

        for i in I:
            m += z[key][i] <= x[i]

        for i in I:
            m += x[i] + x[ind_over] - 1 <= z[key][i]

        for i in I:
            if i == ind_over:
                pass
            else:
                m += v[i] * z[key][i] <= vol_over

    m.optimize()

    selected = [i for i in I if x[i].x >= 0.99]
    print('maximum weigth: {}'.format(c))
    print("selected items: {}".format(", ".join(data[data.index.isin(selected)].Item.to_list())))

In [327]:
data

,Item,Price,Weigth,Volume
0,Laptop,10,0.50,0.20
1,Flashcard,5,0.01,0.02
2,Cryptokey,1,0.01,0.02
3,Lamp,1,5.00,1.00
4,Battery,1,0.05,0.06
5,Charger,0,0.01,0.06


In [328]:
oversized

{'Laptop': 0.05, 'Lamp': 0.01}

In [329]:
vol_knapsack(0.49, data, oversized)

maximum weigth: 0.49
selected items: Flashcard, Cryptokey, Battery


In [330]:
vol_knapsack(0.5, data, oversized)

maximum weigth: 0.5
selected items: Laptop


In [331]:
vol_knapsack(0.51, data, oversized)
vol_knapsack(0.52, data, oversized)
vol_knapsack(0.57, data, oversized)

maximum weigth: 0.51
selected items: Laptop, Flashcard
maximum weigth: 0.52
selected items: Laptop, Flashcard, Cryptokey
maximum weigth: 0.57
selected items: Laptop, Flashcard, Cryptokey


In [332]:
# in simple knapsack we allow to take Battery with 0.57 weigth restriction, 
# but in volume knapsack Battery has larger volume (0.06) than we can take after Laptop (0.05)
simple_knapsack(0.57, data)
vol_knapsack(0.57, data, oversized)

maximum weigth: 0.57
selected items: Laptop, Flashcard, Cryptokey, Battery
maximum weigth: 0.57
selected items: Laptop, Flashcard, Cryptokey


In [333]:
# Even without weigth restriction we can not take Battery in volume knapsack because of
# volume restriction
simple_knapsack(float('Inf'), data)
vol_knapsack(float('Inf'), data, oversized)

maximum weigth: inf
selected items: Laptop, Flashcard, Cryptokey, Lamp, Battery
maximum weigth: inf
selected items: Laptop, Flashcard, Cryptokey
